In [1]:
#import necessary libraries
import numpy as np
import numpy.ma as ma
import os
import astropy.units as u
from astropy.io import ascii
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.stats import sigma_clipped_stats, sigma_clip
import subprocess
from astropy.io import fits
import matplotlib.pyplot as plt
%matplotlib inline
import os

# Move to the data directory for our analysis
os.chdir('NGC6791')

In [2]:
def catalog(w):
    # SDSS Catalog
    from astropy.table import Table
    Q = [Table.read('SDSS-catalog.fit')]

    catalog_coords = SkyCoord(ra=Q[0]['RA_ICRS'], dec=Q[0]['DE_ICRS'], unit='deg')

    catpix_x, catpix_y = w.wcs_world2pix(catalog_coords.ra, catalog_coords.dec, 0)
    mask= (catpix_x>500) & (catpix_x<3500) & (catpix_y>500) & (catpix_y<3500)
    good_cat_stars= Q[0][mask]
    
    return good_cat_stars

In [3]:
def get_table_from_ldac(filename, frame=1):
    """
    Load an astropy table from a fits_ldac by frame (Since the ldac format has column 
    info for odd tables, giving it twce as many tables as a regular fits BinTableHDU,
    match the frame of a table to its corresponding frame in the ldac file).
    
    Parameters
    ----------
    filename: str
        Name of the file to open
    frame: int
        Number of the frame in a regular fits file
    """
    from astropy.table import Table
    if frame>0:
        frame = frame*2
    tbl = Table.read(filename, hdu=frame)
    return tbl

In [4]:
#Function to extract sources
def source_extract(imageName):
    configFile = 'photomCat.sex'
    catalogName = imageName+'.cat'
    paramName = 'photomCat.param'
    try:
        command = 'sex -c %s %s -CATALOG_NAME %s -PARAMETERS_NAME %s' % (configFile, imageName, catalogName, paramName)
        print('Executing command: %s' % command)
        rval = subprocess.run(command.split(), check=True)
    except subprocess.CalledProcessError as err:
        print('Could not run sextractor with exit error %s'%err)
        
    #This is a python wrapper for reading LDAC files produced by SExtractor
    sourceTable = get_table_from_ldac(catalogName)
    
    #filter on the sources to select the ones satisfying our criteria
    cleanSources = sourceTable[(sourceTable['FLAGS']==0) & (sourceTable['FWHM_WORLD'] < 4) & (sourceTable['XWIN_IMAGE']<3500) & (sourceTable['XWIN_IMAGE']>500) &(sourceTable['YWIN_IMAGE']<3500) &(sourceTable['YWIN_IMAGE']>500) &(sourceTable['MAGERR_APER']<1)]
    
    return cleanSources

In [5]:
#Function to calculate zero point calibration
def zero_point(cleanSources, good_cat_stars, band):
    sourceCatCoords = SkyCoord(ra=cleanSources['ALPHAWIN_J2000'], dec=cleanSources['DELTAWIN_J2000'], frame='icrs', unit='degree')
    SDSSCatCoords = SkyCoord(ra=good_cat_stars['RA_ICRS'], dec=good_cat_stars['DE_ICRS'], frame='icrs', unit='degree')

    #Now cross match sources

    #Set the cross-match distance threshold to 0.7 arcsec, or just about one pixel
    photoDistThresh = 0.7
    idx_image, idx_sdss, d2d, d3d = SDSSCatCoords.search_around_sky(sourceCatCoords, photoDistThresh*u.arcsec)
    #idx_image are indexes into sourceCatCoords for the matched sources, while idx_ps1 are indexes into ps1CatCoords for the matched sources

    print('Found %d good cross-matches'%len(idx_image))
    aperture_diameter=10 #set aperture diameter to 10 pixels

    #Array of differences between the catalog and instrumental magnitudes
    offsets = ma.array(good_cat_stars[band][idx_sdss] - cleanSources['MAG_APER'][idx_image])
    #Compute sigma clipped statistics
    zero_mean, zero_med, zero_std = sigma_clipped_stats(offsets)
    zeroDict = {'diameter': aperture_diameter, 'zp_mean': zero_mean, 'zp_median': zero_med, 'zp_std': zero_std}
    return zeroDict

In [6]:
from astropy import units as u
from photutils.aperture import SkyCircularAperture, SkyCircularAnnulus, aperture_photometry
# Background subtraction
from astropy.stats import SigmaClip
from photutils import Background2D, MedianBackground


sigma_clip = SigmaClip(sigma=3) # Sigma clipping
bkg_estimator = MedianBackground()


#function to perform aperture photmetry
def ap_photom(cleanSources, zeroDict, data, w):
    #Background method
    #create empty list to store magnitudes
    mag= []
    
    bkg = Background2D(data, (200, 150), filter_size=(3, 3), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)

    for source in cleanSources:
        ra= source['ALPHAWIN_J2000']
        dec = source['DELTAWIN_J2000']
        #Specify the position of the target in sky coordinates
        position = SkyCoord(ra = ra, dec = dec, unit = u.deg, frame = 'icrs')
        #Now set up a set of apertures to measure flux from
        aperture_radius = 10.0/ 2 #Aperutre radii in pixels 
        aperture = SkyCircularAperture(position, (aperture_radius*0.676/3600)*u.deg)
        #Translate the apertures from sky coordinates to pixel coordinates
        pix_aperture = aperture.to_pixel(w) 
        #Now compute the fluxes within these apertures on our data
        phot_counts = aperture_photometry(data, pix_aperture)
        bg_counts = aperture_photometry(bkg.background, pix_aperture)
    
        #subtract the background flux for this aperture from the source flux
        source_flux = phot_counts['aperture_sum'] - bg_counts['aperture_sum']
        source_error= np.sqrt(phot_counts['aperture_sum']+ bg_counts['aperture_sum'])
    
        source_mag = zeroDict['zp_median'] - 2.5 * np.log10(source_flux)
        source_mag_error = 2.5 / np.log(10) * source_error / source_flux
        mag.append([ra,dec,float(source_mag.value),float(source_mag_error.value)])
        
    return np.array(mag)

#function to perform aperture photmetry
def ap_photom_2(cleanSources, zeroDict, data, w):
    #Annulus Method
    #create empty list to store magnitudes
    mag= []
    anuRadius=10.0
    anuWidth= 10.0

    for source in cleanSources:
        ra= source['ALPHAWIN_J2000']
        dec = source['DELTAWIN_J2000']
        #Specify the position of the target in sky coordinates
        position = SkyCoord(ra = ra, dec = dec, unit = u.deg, frame = 'icrs')
        #Now set up a set of apertures to measure flux from
        aperture_radius = 10.0/ 2 #Aperutre radii in pixels 
        aperture = SkyCircularAperture(position, (aperture_radius*0.676/3600)*u.deg)
        #Translate the apertures from sky coordinates to pixel coordinates
        pix_aperture = aperture.to_pixel(w) 
        #Now compute the fluxes within these apertures on our data
        phot_counts = aperture_photometry(data, pix_aperture)
        
        annulus_aperture = SkyCircularAnnulus(position, r_in = (anuRadius*0.676/3600) * u.deg, r_out = ((anuRadius + anuWidth)*0.676/3600) * u.deg)
        pix_annulus_aperture = annulus_aperture.to_pixel(w)
        annulus_phot_counts = aperture_photometry(data, pix_annulus_aperture)
        #Estimate the background flux per unit pixel area
        bkg_mean = annulus_phot_counts['aperture_sum'] / pix_annulus_aperture.area
    
        #estimate the background flux for this specific aperture size
        bkg_flux = bkg_mean * pix_aperture.area
        #subtract the background flux for this aperture from the source flux
        source_flux = phot_counts['aperture_sum'] - bkg_flux
    
        source_error= np.sqrt(phot_counts['aperture_sum']+ bkg_flux)
    
        source_mag = zeroDict['zp_median'] - 2.5 * np.log10(source_flux)
        source_mag_error = 2.5 / np.log(10) * source_error / source_flux
        mag.append([ra,dec,float(source_mag.value),float(source_mag_error.value)])
        
    return np.array(mag)

def ap_photom_3(cleanSources, zeroDict, data, w):
    #Sextractor Method
    #create empty list to store magnitudes
    mag= []

    for source in cleanSources:
        ra= source['ALPHAWIN_J2000']
        dec = source['DELTAWIN_J2000']
    
        source_mag = zeroDict['zp_median'] + source['MAG_APER']
        source_mag_error = source['MAGERR_APER'] + zeroDict['zp_std']
        mag.append([ra,dec,float(source_mag),float(source_mag_error)])
        
    return np.array(mag)

/var/folders/3w/12g54gn53t1fvjgp87pzg9zc0000gn/T/ipykernel_54864/3195194058.py:5: DeprecationWarning: `photutils.Background2D` is a deprecated alias for `photutils.background.Background2D` and will be removed in the future. Instead, please use `from photutils.background import Background2D` to silence this warning.
  from photutils import Background2D, MedianBackground
/var/folders/3w/12g54gn53t1fvjgp87pzg9zc0000gn/T/ipykernel_54864/3195194058.py:5: DeprecationWarning: `photutils.MedianBackground` is a deprecated alias for `photutils.background.MedianBackground` and will be removed in the future. Instead, please use `from photutils.background import MedianBackground` to silence this warning.
  from photutils import Background2D, MedianBackground


In [7]:
import pandas as pd

data = {'name':['g-10', 'g-40', 'g-120', 'g-240', 'r-10', 'r-40', 'r-120', 'r-240', 'i-10', 'i-40', 'i-120', 'i-240'],
        'band': ['gmag','gmag','gmag','gmag','rmag','rmag','rmag','rmag','imag','imag','imag','imag']}
image_dict = pd.DataFrame(data)

for index, row in image_dict.iterrows():
    imageName = row['name'] + '.fits'
    f = fits.open(imageName)
    data = f[0].data  #This is the image array
    header = f[0].header #This is the header
    
    #store the image WCS into an object
    w = WCS(header)
    
    #call the catalog
    good_cat_stars= catalog(w)
    
    #call sextractror to extract sources
    cleanSources = source_extract(imageName)
    
    #calculate zero point calibration
    zeroDict = zero_point(cleanSources, good_cat_stars, row['band'])
    
    #calculate magnitudes
    magnitudes = ap_photom_3(cleanSources, zeroDict, data, w)
    
    np.savetxt(row['name']+'.csv',magnitudes, delimiter=',')

Executing command: sex -c photomCat.sex g-10.fits -CATALOG_NAME g-10.fits.cat -PARAMETERS_NAME photomCat.param


> 
----- SExtractor 2.25.0 started on 2023-04-27 at 14:29:41 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing Neural Network
> Reading Neural Network Weights
> Initializing catalog
> Looking for g-10.fits
----- Measuring from: g-10.fits
      "NGC6791" / no ext. header / 4096x4108 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line: 1024
> Setting up background map at line: 2048
> Setting up background map at line: 3072
> Setting up background map at line: 4096
> Filtering background map(s)
> Computing background d-map
> Computing background-noise d-map
(M+D) Background: 41.2121    RMS: 10.8252    / Threshold: 21.6504    
> Scanning image
> Line:   25  Objects:        5 detected /        0 sextracted
> Line:   50  Objects:       17 detected /        0 sextracted
> Line:   75  Objects:       21 detected /        0 sextracted
> Line:  100  Objects:       31 detected /        0 sextracted
> L

> Line: 2900  Objects:     1634 detected /      987 sextracted
> Line: 2925  Objects:     1645 detected /     1003 sextracted
> Line: 2950  Objects:     1661 detected /     1023 sextracted
> Line: 2975  Objects:     1682 detected /     1046 sextracted
> Line: 3000  Objects:     1702 detected /     1059 sextracted
> Line: 3025  Objects:     1712 detected /     1071 sextracted
> Line: 3050  Objects:     1729 detected /     1089 sextracted
> Line: 3075  Objects:     1736 detected /     1106 sextracted
> Line: 3100  Objects:     1755 detected /     1124 sextracted
> Line: 3125  Objects:     1776 detected /     1135 sextracted
> Line: 3150  Objects:     1792 detected /     1153 sextracted
> Line: 3175  Objects:     1798 detected /     1172 sextracted
> Line: 3200  Objects:     1810 detected /     1184 sextracted
> Line: 3218  Objects:     1818 detected /     1200 sextracted
> Line: 3225  Objects:     1825 detected /     1205 sextracted
> Line: 3250  Objects:     1835 detected /     1218 sex

Found 370 good cross-matches


Executing command: sex -c photomCat.sex g-40.fits -CATALOG_NAME g-40.fits.cat -PARAMETERS_NAME photomCat.param


...

Found 1571 good cross-matches


Executing command: sex -c photomCat.sex i-240.fits -CATALOG_NAME i-240.fits.cat -PARAMETERS_NAME photomCat.param


> 
----- SExtractor 2.25.0 started on 2023-04-27 at 14:30:44 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing Neural Network
> Reading Neural Network Weights
> Initializing catalog
> Looking for i-240.fits
----- Measuring from: i-240.fits
      "NGC6791" / no ext. header / 4096x4108 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line: 1024
> Setting up background map at line: 2048
> Setting up background map at line: 3072
> Setting up background map at line: 4096
> Filtering background map(s)
> Computing background d-map
> Computing background-noise d-map
(M+D) Background: 352.302    RMS: 18.0029    / Threshold: 36.0057    
> Scanning image
> Line:   25  Objects:       39 detected /        0 sextracted
> Line:   50  Objects:       84 detected /        0 sextracted
> Line:   75  Objects:      139 detected /        0 sextracted
> Line:  100  Objects:      202 detected /        0 sextracted
>

> Line: 2400  Objects:    10864 detected /     3345 sextracted
> Line: 2425  Objects:    11004 detected /     3431 sextracted
> Line: 2450  Objects:    11136 detected /     3526 sextracted
> Line: 2468  Objects:    11235 detected /     3600 sextracted
> Line: 2475  Objects:    11263 detected /     3628 sextracted
> Line: 2500  Objects:    11396 detected /     3721 sextracted
> Line: 2525  Objects:    11511 detected /     3820 sextracted
> Line: 2550  Objects:    11644 detected /     3920 sextracted
> Line: 2571  Objects:    11719 detected /     4000 sextracted
> Line: 2575  Objects:    11730 detected /     4025 sextracted
> Line: 2600  Objects:    11901 detected /     4149 sextracted
> Line: 2625  Objects:    12046 detected /     4271 sextracted
> Line: 2650  Objects:    12152 detected /     4390 sextracted
> Line: 2653  Objects:    12170 detected /     4400 sextracted
> Line: 2675  Objects:    12250 detected /     4510 sextracted
> Line: 2700  Objects:    12378 detected /     4609 sex

Found 2284 good cross-matches
